# Step 1: Environment Setup and Dependency Installation

To start, we'll set up the environment and ensure that all required dependencies are installed. We'll use a `requirements.txt` file, which specifies the exact packages and versions needed for the project, ensuring consistency across environments.

## 1.1 Install Dependencies from `requirements.txt`

Run the code below to install all necessary dependencies. This command will read from `requirements.txt` and install each package listed:

```python
# Install dependencies
!pip install -r requirements.txt


# Step 2: Preparation of Historical Data from CEX and DeFi

In this section, we'll prepare historical data from centralized exchanges (CEX) and decentralized finance (DeFi) platforms. Specifically, we will pull data from Aave Lending for DeFi information and Binance Trading for CEX data.

To facilitate this, we will use the `TokenDataFinder` class from the `data_prepare.py` module, which automates the process of locating and extracting time ranges from CSV files in specified folders.

## 2.1 Overview of `TokenDataFinder` Class

The `TokenDataFinder` class is designed to:
1. **Locate CSV Files**: It searches the specified folders for CSV files containing historical data.
2. **Extract Date Ranges**: It reads the first and last rows of each CSV file to determine the range of available data.
3. **Identify Data Paths**: It differentiates between data from Aave Lending and Binance Trading (both SPOT and PERP markets).

## Code Explanation

Below is the code for `TokenDataFinder` from `data_prepare.py`. We’ll use its methods to gather data for each token across Aave Lending and Binance Trading platforms.


In [1]:
# Install dependencies
# !pip install -r requirements.txt

# Import the TokenDataFinder class from data_prepare module
from data_prepare import TokenDataFinder

# Define the folder paths for Aave Lending and Binance Trading data
aave_lending_folder = 'aave_lending'
binance_trading_folder = 'binance_trading'

# Initialize the TokenDataFinder with the specified folder paths
finder = TokenDataFinder(aave_lending_folder, binance_trading_folder)

# Use the find_csv_files method to get paths and date ranges for each token
token_paths = finder.find_csv_files()

# Print the results for each token, including paths and date ranges
for token, paths in token_paths.items():
    print(f"Token: {token}")

    # Print Aave Lending data path and date range
    print(
        f"  Aave Lending Path: {paths['aave_lending']['path']}\n Data From: {paths['aave_lending']['data_from']} Data To: {paths['aave_lending']['data_to']}"
    )

    # Check if Binance Spot Trading data is available and print path and date range
    if 'binance_trading_spot' in paths:
        print(
            f"  Binance Trading Spot Path: {paths['binance_trading_spot']['path']}\n Data From: {paths['binance_trading_spot']['data_from']} Data To: {paths['binance_trading_spot']['data_to']}"
        )
    else:
        print("  Binance Trading Spot Path: Not Found")

    # Print the 'lan' value, which indicates the token's folder for Aave lending data
    print(f"  Lan: {paths.get('lan')}\n")

Error reading aave_lending\polygon\AAVE.csv: list index out of range
Token: AVAX
  Aave Lending Path: aave_lending\avalanche\AVAX.csv
 Data From: 2024-09-21 22:00:00 Data To: 2024-10-06 21:00:00
  Binance Trading Spot Path: binance_trading\SPOT\AVAX-USDT\1MIN\data.csv
 Data From: 2024-09-24 08:46:00 Data To: 2024-10-01 08:45:00
  Lan: avalanche

Token: BTC
  Aave Lending Path: aave_lending\mainnet\BTC.csv
 Data From: 2024-09-21 22:00:00 Data To: 2024-10-06 21:00:00
  Binance Trading Spot Path: binance_trading\SPOT\BTC-USDT\1MIN\data.csv
 Data From: 2024-09-24 08:36:00 Data To: 2024-10-01 08:35:00
  Lan: mainnet

Token: ETH
  Aave Lending Path: aave_lending\mainnet\ETH.csv
 Data From: 2024-09-21 22:00:00 Data To: 2024-10-06 22:00:00
  Binance Trading Spot Path: binance_trading\SPOT\ETH-USDT\1MIN\data.csv
 Data From: 2024-09-24 08:37:00 Data To: 2024-10-01 08:36:00
  Lan: mainnet

Token: LINK
  Aave Lending Path: aave_lending\mainnet\LINK.csv
 Data From: 2024-09-21 22:00:00 Data To: 2024

# Step 3: Preparation of Historical Data from CEX and DeFi

In this step, we will select tokens with a significant volume of information based on existing data. We will then define the backtesting period during which we will collect relevant news articles and analyze them using a Large Language Model (LLM). 

> **Note:** Various models and prompts were tested to determine the most effective approach for analysis.

> **Note:** The module from which the procedures are imported and executed is located in the main directory. This is where we invoke the relevant methods.

> **Note:** News will be collected from NewsAPI, and the analysis will be performed using the OpenAI model. Therefore, to run this code independently, you will need to create a file with the API keys named `api_key.env`. The structure of the file should be as follows:
OPENAI_API_KEY=[YOUR_OPENAI_KEY]
NEWS_API_KEY=[YOUR_NEWS_API_KEY]


In [3]:
import requests
import urllib.parse
import pandas as pd
import os, json
import time
from datetime import datetime, timedelta
import openai
from dotenv import load_dotenv

# Import the CryptoNewsAnalyzer class from get_news.py module
from get_news import CryptoNewsAnalyzer

# Load environment variables from api.env
# OPENAI_API_KEY=sk-proj-NvmRe*****************
# NEWS_API_KEY=b5*******************
load_dotenv("api_key.env")

# Instantiate the OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

api_key = os.getenv("NEWS_API_KEY")

# Define the tokens, start date, and end date for news analysis
tokens = ["BTC", "ETH", "LINK"]

# Set the backtesting period
start_date = datetime(2024, 9, 27)
end_date = datetime(2024, 10, 6)

# Initialize the news analyzer with tokens, API key, and date range
news_analyzer = CryptoNewsAnalyzer(tokens, api_key, start_date, end_date)

# Fetch news for each token within the specified date range
for token in tokens:
    news_analyzer.get_news(token)

# Analyze the sentiment of the collected news articles
news_analyzer.analyze_news_sentiment()

# Save the sentiment analysis results to CSV for future reference
news_analyzer.save_sentiment_to_csv()

# Further analyze the news data to determine significant rise/fall events
news_analyzer.analyze_news()


Using existing data from ./BTC\2024-09-27_to_2024-10-06_everything_BTC.csv
Using existing data from ./ETH\2024-09-27_to_2024-10-06_everything_ETH.csv
Using existing data from ./LINK\2024-09-27_to_2024-10-06_everything_LINK.csv
Loading existing sentiment data from ./BTC\2024-09-27_to_2024-10-06_sentiment_BTC.csv
Loading existing sentiment data from ./ETH\2024-09-27_to_2024-10-06_sentiment_ETH.csv
Loading existing sentiment data from ./LINK\2024-09-27_to_2024-10-06_sentiment_LINK.csv
Using existing sentiment data from ./BTC\2024-09-27_to_2024-10-06_sentiment_BTC.csv
Using existing sentiment data from ./ETH\2024-09-27_to_2024-10-06_sentiment_ETH.csv
Using existing sentiment data from ./LINK\2024-09-27_to_2024-10-06_sentiment_LINK.csv
Token: BTC
Total news: 100
Total rise impact news: 16
Rise impact news period: 2024-09-27T03:00:53Z to 2024-10-05T17:30:47Z
Rise impact news dates (sorted): ['2024-09-27T03:00:53Z', '2024-09-27T06:40:10Z', '2024-09-27T09:51:00Z', '2024-09-27T12:01:00Z', '2024

## Analysis Results and Methodology

The output from the code execution indicates that historical news data and sentiment analysis have been successfully loaded or processed for each token (BTC, ETH, LINK) within the specified backtesting period (2024-09-27 to 2024-10-06).

### Summary of Results
For each token, two main types of impact events are identified and classified:

1. **Rise Impact News**:
   - This category includes news events that are deemed to have a "significant rise" impact on the token’s price.
   - These news articles are identified based on sentiment analysis, where the content suggests a potential positive effect on the token’s future price.
   - For instance, BTC had 16 such rise events within this period, indicating frequent positive sentiment articles that may influence its price in the near term.
   
2. **Fall Impact News**:
   - This category includes news events that are classified as having a "fall" impact on the token’s price.
   - These articles, as determined by the sentiment analysis, imply a potentially negative effect on the token’s future price.
   - For example, LINK had 22 fall impact news events within the same period, signaling the presence of considerable negative sentiment.

### Key Methodology and Criteria
The identification and classification of news into "Rise" and "Fall" categories are based on a combination of sentiment analysis and historical context. This approach introduces an **innovative filtering mechanism** that separates impactful news from retrospective reports, enhancing the relevance and actionability of the insights.

1. **News Sentiment Evaluation**:
   - Each news article is processed using a custom prompt in the OpenAI model, which categorizes sentiment into five levels: "significant rise," "rise," "neutral," "fall," and "significant fall."
   - Only articles with a "significant rise" or "fall" sentiment are selected to populate the rise and fall events lists, respectively, ensuring that only the most impactful news is considered.

2. **Filtering Retrospective News**:
   - An innovative aspect of this methodology is the filtering of news that contains primarily retrospective information, such as reports that BTC has already reached a specific level. These retrospective news articles are less likely to impact future price movements and are therefore excluded, ensuring that only forward-looking insights contribute to the backtesting results.
   - This selective approach not only improves the quality of the backtesting process but also creates opportunities for **real-time response**, a critical capability in modern trading and DeFi environments.

3. **Agentic Capabilities**:
   - By capturing only the most relevant forward-looking news, this method aligns well with the requirements for deploying autonomous agents, such as **Atlas, Ganesh, or Apollo**, that can interact with DeFi primitives and LLMs. This enables the development of agents capable of real-time adaptation to sentiment shifts, making the strategy highly responsive and effective in dynamic market conditions.

4. **Deduplication and Skipping**:
   - If the relevant data already exists (for example, in sentiment analysis CSVs or rise/fall JSON files), the code skips reprocessing to ensure efficiency and consistency in results.

### Implications for Backtesting
The rise and fall impact dates stored in JSON files provide a list of key sentiment-driven events, which can be used to simulate the influence of news on the tokens during the backtesting period. By focusing on news with a future-oriented impact, this methodology enables a more realistic testing environment for evaluating trading strategies based on external news sentiment. This approach also lays a strong foundation for building robust agent-based strategies, paving the way for a pseudocode implementation that could interact with DeFi primitives, LLMs, and on-chain data sources for comprehensive market analysis and strategic decision-making.


## Step 4: Backtesting Strategy and Methodology

Given the limited backtesting period and specific news volume per token, the focus was on a primary trading pair with an effective yet straightforward trading strategy. This approach facilitated robust analysis of market reactions to sentiment signals, liquidity data, and technical indicators.

### Methodology
The methodology involved testing multiple strategies to achieve a balance between simplicity and effectiveness. An optimal strategy was selected based on the following components:

1. **Strategy Design**:
   - The strategy enters a buy position when a token has a "Rise Impact" news signal, indicating potential upward price movement.
   - After entering a position, the strategy monitors the token's price using predefined exit conditions to minimize downside risk while maximizing upside potential.

2. **Exit Conditions**:
   - A position is closed when:
     - A "Fall Impact" news signal is detected, suggesting a potential negative price movement.
     - The price reaches the upper Bollinger Band on a 1-hour timeframe, adjusted by a model parameter defining the take-profit level.
     - The price falls to a Stop Loss level calculated based on the lower Bollinger Band.

3. **Bollinger Bands and Adaptive Parameters**:
   - Bollinger Bands are utilized to dynamically set take-profit and stop-loss levels based on market volatility.
   - Parameters (`sl_level` and `tp_level`) allow for adjustments to Bollinger Band levels, enabling flexibility in response to market conditions.

### Backtesting Process
The backtesting is conducted using the Backtrader library, which efficiently handles historical data and real-time backtesting capabilities. Key steps include:

1. **Data Preparation**:
   - Historical data from CEX and DeFi platforms is pre-processed for consistency across timestamps and value formats.
   - Pre-aggregated news impact dates and sentiment are loaded for each token, simulating realistic trading signals.

2. **Strategy Initialization**:
   - The strategy initializes with a starting balance and parameters for position sizing, precision, and the backtesting period.
   - Sentiment data (Rise and Fall Impact News) is loaded for each token to ensure the strategy responds to relevant events.

3. **Execution and Monitoring**:
   - The strategy enters buy positions based on "Rise Impact" news signals and monitors the market using Bollinger Bands and stop-loss conditions.
   - Each position's metrics (entry/exit prices, portfolio value) are tracked for performance evaluation.

4. **Performance Evaluation**:
   - Backtesting results include key metrics such as Total Trades, Win/Loss ratio, Gross/Net Profit, and Sharpe Ratio.
   - These metrics evaluate the strategy’s effectiveness and risk-adjusted returns.

### Real-World Applications
This strategy demonstrates an innovative approach by leveraging sentiment signals while filtering out retrospective news events. This allows for real-time responsiveness and supports the development of agents such as **Atlas, Ganesh, or Apollo**, which can interact with DeFi primitives and LLMs for refined trading decisions.

**Note:** Currently, the console outputs summarized information for the primary token. However, the code allows full backtesting for other tokens given sufficient data and a larger timeframe.

**Note:** While this strategy shows excellent results, it's important to recognize that for a more relevant evaluation of the Sharpe Ratio and other metrics like the Sortino Ratio, testing over longer timeframes is necessary. This requires additional work and potentially purchasing paid API versions, including news sources.


In [4]:
# Import the necessary classes from the backtesting_program module
from backtesting_program import BacktestingProgram

# Define the token to be backtested
for token in ["BTC"]:  # Extendable for additional tokens as needed
    backtesting_period = ['2024-09-27', '2024-10-06']  # Define the backtesting period

    # Initialize the BacktestingProgram instance for the specified token
    backtesting_program = BacktestingProgram(
        _backtesting_period=backtesting_period,  # Set the backtesting period
        precision_qty=4,  # Define precision for quantity
        precision_price=2,  # Define precision for price
        pair=token,  # Set the token to be tested
    )

    # Execute the backtesting process
    backtesting_program.run()  # Run the backtesting algorithm

    # Output the results of the backtesting
    backtesting_program.print_backtesting_results()  # Print the final results to the console


Loaded and rounded rise dates from ./BTC\2024-09-27_to_2024-10-06_rise_dates_BTC.json
Loaded and rounded fall dates from ./BTC\2024-09-27_to_2024-10-06_fall_dates_BTC.json
[TradingStrategy] Executed BUY order for 0.1541 at price 64862.71 on 2024-09-27 03:01:00
[TradingStrategy] Executed SELL order for -0.1541 at price 65221.76 on 2024-09-27 05:27:00
[TradingStrategy] After order: Cash Balance: 10055.33 USD, Position: 0.0000 tokens (Value: 0.0 USD)

[TradingStrategy] Executed BUY order for 0.1535 at price 65465.12 on 2024-09-27 06:41:00
[TradingStrategy] Executed SELL order for -0.1535 at price 65690.01 on 2024-09-27 08:25:00
[TradingStrategy] After order: Cash Balance: 10089.85 USD, Position: 0.0000 tokens (Value: 0.0 USD)

[TradingStrategy] Executed BUY order for 0.1533 at price 65806.42 on 2024-09-27 09:52:00
[TradingStrategy] Executed SELL order for -0.1533 at price 65795.97 on 2024-09-27 09:53:00
[TradingStrategy] After order: Cash Balance: 10088.25 USD, Position: 0.0000 tokens (Va

# Conclusion

In summary, we have developed a comprehensive backtesting framework leveraging an order book and historical financial datasets to create a detailed analysis of our trading strategy. The primary focus was on data aggregation, strategy design, and backtesting results, with future enhancements to allow for the development of agents like **Atlas**, **Ganesh**, or **Apollo**.

## Key Findings

Our backtesting results provide valuable insights into the effectiveness of the Trading Strategy implemented. The significant metrics obtained from the testing period are as follows:

- **Final Portfolio Value**: $10,170.57
- **PnL**: 1.71%
- **Price Change for the Period**: -1.85% (from $65,173.99 to $63,967.99)
- **Total Trades**: 6
- **Closed Trades**: 6
- **Gross PnL**: $170.57
- **Net PnL**: $170.57
- **Wins**: 4
- **Losses**: 2
- **Sharpe Ratio**: 0.41

Despite a market downturn reflected by a **Price Change** of -1.85%, the strategy yielded a positive **PnL** of 1.71%. This outcome is particularly noteworthy given that it was achieved over a short testing period. Furthermore, a **Sharpe Ratio** of 0.41 suggests that the strategy performed relatively well against the risks undertaken.

## Future Potential

The methodology employed in this strategy highlights the potential for developing innovative, real-time algorithms capable of reacting dynamically to market changes. By continuously monitoring news and market sentiment, we can enhance the decision-making process, enabling traders to capitalize on timely information. This not only augments the existing trading strategy but also lays the groundwork for creating advanced agents that can interact with DeFi primitives and integrate seamlessly with LLMs.

Additionally, the findings and analyses from this notebook have been exported to `flash_news.html` for ease of review and analysis, allowing stakeholders to quickly access key insights and results.

In conclusion, our approach demonstrates the efficacy of combining historical data analysis with real-time news aggregation to develop strategies that are not only robust but also adaptable to the ever-changing landscape of cryptocurrency trading.






